In [13]:
import os
import pdb
import argparse
import pickle as pkl
import tensorflow as tf

from collections import defaultdict

import numpy as np
import matplotlib as mpl
mpl.use('Agg')
import matplotlib.pyplot as plt

from easydict import EasyDict

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

# Local imports
import utils
import data_handler
from data_handler import *


In [162]:
# Options/ Hyperparameters required to train and test the model
opts = EasyDict()

opts.n_epochs = 1000
opts.batch_size = 32
opts.learning_rate = 0.0001
opts.lr_decay = 0.99
opts.hidden_layer_size = 100
opts.model_name = "simple_rnn"
opts.checkpoints_dir = "./checkpoints/"+opts.model_name 

TEST_SENTENCE = 'i love deep learning'

In [163]:
utils.create_dir_if_not_exists(opts.checkpoints_dir)

In [164]:
line_pairs, vocab_size, idx_dict = load_data()

In [165]:
# dividing the line pairs into 8:2, train and val split
num_lines = len(line_pairs)
num_train = int(0.8 * num_lines)
train_pairs, val_pairs = line_pairs[:num_train], line_pairs[num_train:]

In [166]:
train_dict = create_dict(train_pairs)
val_dict = create_dict(val_pairs)

# Study the structure of the created train_dict and val_dict variables

In [179]:
class MyGRUCell(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(MyGRUCell, self).__init__()
        #TODO: zero intit, weight return
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.weight_z=nn.Linear(input_size + hidden_size , hidden_size)
        self.weight_r=nn.Linear(input_size + hidden_size , hidden_size)
        self.weight = nn.Linear(input_size + hidden_size , hidden_size)
        #self.weight_z = torch.zeros(input_size + hidden_size , hidden_size, requires_grad=True)
        #torch.nn.init.xavier_uniform_(self.weight_z)
        #self.weight_r = torch.zeros(input_size + hidden_size , hidden_size, requires_grad=True)
        #torch.nn.init.xavier_uniform_(self.weight_r)
        #self.weight = torch.zeros(input_size + hidden_size , hidden_size, requires_grad=True)
        #torch.nn.init.xavier_uniform_(self.weight)
        #bias
        #self.bias_z = torch.zeros(1 , hidden_size, requires_grad=True)
        #torch.nn.init.xavier_uniform_(self.bias_z,)
        #self.bias_r = torch.zeros(1 , hidden_size,requires_grad=True)
        #torch.nn.init.xavier_uniform_(self.bias_r)
        #self.bias = torch.zeros(1 , hidden_size,requires_grad=True)
        #torch.nn.init.xavier_uniform_(self.bias)
    
        # ------------
        # FILL THIS IN
        # ------------
        
    def forward(self, x, h_prev):
        """Forward pass of the GRU computation for one time step.

        Arguments
            x: batch_size x input_size
            h_prev: batch_size x hidden_size

        Returns:
            h_new: batch_size x hidden_size
        """
        #print h_prev.shape , x.shape, "thiiiiis"
        input_concatenated = torch.cat((h_prev , x), 1)
        z = torch.sigmoid(self.weight_z(input_concatenated))
        r = torch.sigmoid(self.weight_r(input_concatenated))
        g = torch.tanh(self.weight(torch.cat(((r*h_prev),x),1)))
        h_new = ((1-z)*h_prev)+ (z*g)

        # ------------
        # FILL THIS IN
        # ------------
        # z = ...
        # r = ...
        # g = ...
        # h_new = ...
        return h_new 
   # def parameters(self):
       # return[self.weight_z,self.weight_r, self.weight]

# Implement your Encoder RNN using instances of GRU Cell you just created.
# You would need a character embedding layer for this.
# Implement your Encoder RNN using instances of GRU Cell you just created.
# You would need a character embedding layer for this.

class Encoder(nn.Module):
    def __init__(self, vocab_size, hidden_size):
        super(Encoder, self).__init__()

        self.vocab_size = vocab_size
        self.hidden_size = hidden_size
        self.gru = torch.nn.GRU(self.vocab_size, self.hidden_size, batch_first=True, num_layers = 1, bidirectional = True)
        #self.gru = torch.nn.GRU(self.vocab_size, self.hidden_size, bidirectional = True)
    
        # embedding layer
        # GRU cell

    def forward(self, inputs):
        """Forward pass of the encoder RNN.

        Arguments:
            inputs: Input token indexes across a batch for all time steps in the sequence. (batch_size x seq_len)

        Returns:
            annotations: The hidden states computed at each step of the input sequence. (batch_size x seq_len x hidden_size)
            hidden: The final hidden state of the encoder, for each sequence in a batch. (batch_size x hidden_size)
        """



        batch_size, seq_len = inputs.size()
        inp_onehot = torch.FloatTensor(batch_size, self.vocab_size, seq_len)
        inp_onehot.zero_()
        #print(inputs.size())
        #print(inp_onehot.size())
        #print(torch.unsqueeze(inputs,1).size())
        embed=inp_onehot.scatter_(1, (torch.unsqueeze(inputs,1).type(torch.LongTensor)), 1)
        embed_unstack = torch.unbind(embed,2)
        hidden = self.init_hidden(batch_size)
        annotations = []
        for i in range (seq_len):
            #print i
            #print embed_unstack[i].shape , hidden.shape
            #print embed_unstack[i].size() , "embed_unstack"
            embed_unstack_input = torch.Tensor(embed_unstack[i]).unsqueeze(1)
            #print batch_size
            #print self.hidden_size
            #print hidden.size(),"hidden_size"
            #print embed_unstack_input.size() , "embed input size"
            _ , hidden = self.gru(embed_unstack_input, hidden)
            annotations.append(hidden)
        #print (annotations.size())
            
        # The encoded embeddings should be of size batch_size x seq_len x hidden_size
        # Complete the forward pass 
        # ....
        
        return annotations, hidden

    def init_hidden(self, bs):
        """Creates a tensor of zeros to represent the initial hidden states
        of a batch of sequences.

        Arguments:
            bs: The batch size for the initial hidden state.

        Returns:
            hidden: An initial hidden state of all zeros. (batch_size x hidden_size)
        """
        return torch.zeros(2,bs, self.hidden_size)
   # def parameters(self):
       # return self.cell.parameters()

# Implement your Decoder RNN using instances of GRU Cell you just created.
# You would need a character embedding layer for this. 
# In addition you would also require an activation function applied to the output of the GRU Cell

class Decoder(nn.Module):
    def __init__(self, vocab_size, hidden_size):
        super(Decoder, self).__init__()
        self.vocab_size = vocab_size
        self.hidden_size = hidden_size
        #self.weight = torch.zeros(hidden_size , vocab_size, requires_grad=True)
        #torch.nn.init.xavier_uniform_(self.weight)
        #self.bias = torch.zeros(1 , vocab_size, requires_grad=True)
        #torch.nn.init.xavier_uniform_(self.bias)
        self.weight= nn.Linear(hidden_size,vocab_size)
        self.gru = torch.nn.GRU(self.vocab_size, self.hidden_size, bidirectional = True,  batch_first=True, num_layers = 1)
        
        # define embedding layer
        # define GRU cell
        # define out

    def forward(self, x, h_prev):
        """Forward pass of the decoder RNN.

        Arguments:
            x: Input token indexes across a batch for a single time step. (batch_size x 1)
            h_prev: The hidden states from the previous step, across a batch. (batch_size x hidden_size)

        Returns:
            output: Un-normalized scores for each token in the vocabulary, across a batch. (batch_size x vocab_size)
            h_new: The new hidden states, across a batch. (batch_size x hidden_size)
        """
        batch_size = x.size(0)
        inp_onehot = torch.FloatTensor(batch_size, self.vocab_size)
        inp_onehot.zero_()
        #print(inp_onehot.size())
        #print("x size", x.size())
        embed=inp_onehot.scatter_(1, x.type(torch.LongTensor), 1)
        embed=torch.Tensor(embed).unsqueeze(1)
        #print h_prev.size()
        #print("embed size" ,embed.size())
        _,h_new = self.gru(embed,h_prev)
        out = h_new[:][0][:]
        out= out.squeeze(1)
        #print out.size()
        output = self.weight(out) ##check this!
        # Implement the forward pass of this network
        # ....
        return output, h_new
   # def parameters(self):
    #    return self.cell.parameters(), [self.weight, self.bias]

In [180]:
##########################################################################
### Setup: Create Encoder, Decoder Objects ###
##########################################################################
encoder = Encoder(vocab_size=vocab_size, hidden_size=opts.hidden_layer_size)
decoder = Decoder(vocab_size=vocab_size, hidden_size=opts.hidden_layer_size)


In [181]:
def train_model(train_dict, val_dict, idx_dict, encoder, decoder, opts):
    """Runs the main training loop; evaluates the model on the val set every epoch.
        * Prints training and val loss each epoch.
        * Prints qualitative translation results each epoch using TEST_SENTENCE

    Arguments:
        train_dict: The training word pairs, organized by source and target lengths.
        val_dict: The validation word pairs, organized by source and target lengths.
        idx_dict: Contains char-to-index and index-to-char mappings, and start & end token indexes.
        encoder: An encoder model to produce annotations for each step of the input sequence.
        decoder: A decoder model to generate output tokens.
        opts: The input arguments for hyper-parameters and others.
    """

    criterion = nn.CrossEntropyLoss()
    params_enc=encoder.parameters()
    params_dec=decoder.parameters()
    #print(params.size())
    #decoder_param1= decoder.parameters()
    #print(encoder1.parameters().size())
    optimizer = optim.Adam(list(params_enc)+list(params_dec), lr=opts.learning_rate) 

    start_token = idx_dict['start_token']
    end_token = idx_dict['end_token']
    char_to_index = idx_dict['char_to_index']

    loss_log = open(os.path.join(opts.checkpoints_dir, 'loss_log.txt'), 'w')

    best_val_loss = 1e6
    train_losses = []
    val_losses = []

    key=(11, 16)
    #print(train_dict[key])
    train_dict2 = {key: train_dict[key]}
    for epoch in range(opts.n_epochs ):

            # decay the learning rate of the optimizer
            # ....
    #         optimizer.param_groups[0]['lr'] *= opts.lr_decay
        optimizer.param_groups[0]['lr'] *= opts.lr_decay
        epoch_losses = []
        for key in train_dict:
            #print (train_dict[key])
            #print(key)
            #print(train_dict2[key])

            input_strings, target_strings = zip(*train_dict[key])

            # Make your input tensor and the target tensors
            # HINT : use the function string_to_index_list given in data_handler.py
            # input_tensors = ....
            # output_tensors = ....
            input_tensors = [data_handler.string_to_index_list(string, char_to_index, end_token) for string in input_strings]
            output_tensors = [data_handler.string_to_index_list(string, char_to_index, end_token) for string in target_strings]
            input_tensors = torch.Tensor(input_tensors)
            output_tensors = torch.Tensor(output_tensors)
            num_tensors = len(input_tensors)
            num_batches = int(np.ceil(num_tensors / float(opts.batch_size)))

            for i in range(num_batches):
                #print (i)
                start = i * opts.batch_size
                end = start + opts.batch_size

                # Define inputs and targets for THIS batch, beginning at index 'start' to 'end'
                # inputs = ....
                # outputs = ....
                inputs = torch.Tensor(input_tensors[start:end])
                outputs = torch.Tensor(output_tensors[start:end])
                # The batch size may be different in each epoch
                BS = inputs.size(0)

                encoder_annotations, encoder_hidden = encoder.forward(inputs)

                # The last hidden state of the encoder becomes the first hidden state of the decoder
                # decoder_hidden = ....
                decoder_hidden = encoder_hidden
                # Define the first decoder input. This would essentially be the start_token
                # decoder_input = ....
                decoder_input = torch.Tensor([start_token for i in range(BS)])
                #print("decoder sized 1",decoder_input.size())
                decoder_input = decoder_input.unsqueeze(1)
                #print("decoder sized",decoder_input.size())
                loss = 0.0

                seq_len = outputs.size(1)  # Gets seq_len from BS x seq_len
                for i in range(seq_len):
                    #print ("this" , i)
                    #print(decoder_hidden.size())
                    decoder_output, decoder_hidden = decoder.forward(decoder_input, decoder_hidden)
                    #print (decoder_output.size())
                    current_target = outputs[:,i]
                    #print(current_target)
                            # Calculate the cross entropy between the decoder distribution and Ground truth (current_target)
                            # loss += ....
                    #print(criterion(decoder_output, current_target.type(torch.LongTensor)))
                    loss += criterion(decoder_output, current_target.type(torch.LongTensor))
                            # Find out the most probable character (ni) from the softmax distribution produced
                            # ni = ....

                    decoder_input = outputs[:,i].unsqueeze(1)

                loss /= float(seq_len)
                #print(loss, "loss")
                epoch_losses.append(loss.item())

                        # Compute gradients
                loss.backward()

                        # Update the parameters of the encoder and decoder
                optimizer.step()

        train_loss = np.mean(epoch_losses)
        val_loss = evaluate(val_dict, encoder, decoder, idx_dict, criterion, opts)

        if val_loss < best_val_loss:
            utils.store_checkpoints(encoder, decoder, idx_dict, opts)

        gen_string = find_pig_latin(TEST_SENTENCE, encoder, decoder, idx_dict, opts)
        #gen_string="hello"
        print("Epoch: {:3d} | Train loss: {:.3f} | Val loss: {:.3f} | Gen: {:20s}".format(epoch, train_loss, val_loss, gen_string))

        loss_log.write('{} {} {}\n'.format(epoch, train_loss, val_loss))
        loss_log.flush()

        train_losses.append(train_loss)
        val_losses.append(val_loss)

        utils.store_loss_plots(train_losses, val_losses, opts)




In [182]:
def evaluate(data_dict, encoder, decoder, idx_dict, criterion, opts):
    """Evaluates the model on a held-out validation or test set. 
    This should be pretty straight-forward if you have figured out how to do the training correctly.
    From then, it's just copy and paste.

    Arguments:
        data_dict: The validation/test word pairs, organized by source and target lengths.
        encoder: An encoder model to produce annotations for each step of the input sequence.
        decoder: A decoder model to generate output tokens.
        idx_dict: Contains char-to-index and index-to-char mappings, and start & end token indexes.
        criterion: Used to compute the CrossEntropyLoss for each decoder output.
        opts: The command-line arguments.

    Returns:
        mean_loss: The average loss over all batches from data_dict.
    """

    start_token = idx_dict['start_token']
    end_token = idx_dict['end_token']
    char_to_index = idx_dict['char_to_index']

    losses = []

    for key in data_dict:

        input_strings, target_strings = zip(*data_dict[key])
        # Make your input tensor and the target tensors
        # HINT : use the function string_to_index_list given in data_handler.py
        # input_tensors = ....
        # output_tensors = ....
        input_tensors = [data_handler.string_to_index_list(string, char_to_index, end_token) for string in input_strings]
        output_tensors = [data_handler.string_to_index_list(string, char_to_index, end_token) for string in target_strings]
        input_tensors = torch.Tensor(input_tensors)
        output_tensors = torch.Tensor(output_tensors)
        num_tensors = len(input_tensors)
        num_batches = int(np.ceil(num_tensors / float(opts.batch_size)))

        for i in range(num_batches):

            start = i * opts.batch_size
            end = start + opts.batch_size

            # Define inputs and targets for THIS batch, beginning at index 'start' to 'end'
            # inputs = ....
            # outputs = ....
            inputs = torch.Tensor(input_tensors[start:end])
            outputs = torch.Tensor(output_tensors[start:end])
            # The batch size may be different in each epoch
            BS = inputs.size(0)

            encoder_annotations, encoder_hidden = encoder.forward(inputs)
            
            # The last hidden state of the encoder becomes the first hidden state of the decoder
            # decoder_hidden = ....
            decoder_hidden = encoder_hidden
            # Define the first decoder input. This would essentially be the start_token
            # decoder_input = ....
            decoder_input = torch.Tensor([start_token for i in range(BS)])
            #print("decoder sized 1",decoder_input.size())
            decoder_input = decoder_input.unsqueeze(1)

            loss = 0.0

            seq_len = outputs.size(1)  # Gets seq_len from BS x seq_len

            for i in range(seq_len):
                decoder_output, decoder_hidden= decoder.forward(decoder_input, decoder_hidden)

                current_target = outputs[:,i]
                #print decoder_output.size()
                # Calculate the cross entropy between the decoder distribution and Ground truth (current_target)
                # loss += ....
                loss += criterion(decoder_output, current_target.type(torch.LongTensor))
                # Find out the most probable character (ni) from the softmax distribution produced
                # ni = ....
                
                # Update decoder_input at the next time step to be this time-step's target 
                # decoder_input = ....
                decoder_output=torch.softmax(decoder_output.float(),1)
                _, indices = torch.max(decoder_output, 1)
                #indices=torch.unsqueeze(indices,1)
                #inp_onehot = torch.FloatTensor(BS, vocab_size)
                #inp_onehot.zero_()
                #print(inp_onehot.size())
                #print("x size", x.size())
                #decoder_input=inp_onehot.scatter_(1, indices.type(torch.LongTensor), 1)
                decoder_input = outputs[:,i].unsqueeze(1)
                
            loss /= float(seq_len)
            losses.append(loss.item())

    mean_loss = np.mean(losses)

    return mean_loss


In [183]:
def find_pig_latin(sentence, encoder, decoder, idx_dict, opts):
    """Translates a sentence from English to Pig-Latin, by splitting the sentence into
    words (whitespace-separated), running the encoder-decoder model to translate each
    word independently, and then stitching the words back together with spaces between them.
    """
    return ' '.join([translate(word, encoder, decoder, idx_dict, opts) for word in sentence.split()])


def translate(input_string, encoder, decoder, idx_dict, opts):
    """Translates a given string from English to Pig-Latin.
    Not much to do here as well. Follows basically the same structure as that of the function evaluate.
    """

    char_to_index = idx_dict['char_to_index']
    index_to_char = idx_dict['index_to_char']
    start_token = idx_dict['start_token']
    end_token = idx_dict['end_token']

    max_generated_chars = 20
    gen_string = ''
    #print input_string
    # convert given string to an array of indexes
    # HINT: use the function string_to_index_list provided in data_handler
    # indexes = ....
    indexes = data_handler.string_to_index_list(input_string, char_to_index, end_token)
    #print(input_string)
    indexes =(torch.Tensor(indexes))
    indexes =indexes.unsqueeze(0)
    #print(indexes.size(), "index size")
    encoder_annotations, encoder_last_hidden = encoder.forward(torch.Tensor(indexes))
    #print(torch.Tensor(encoder_last_hidden).size())
    # The last hidden state of the encoder becomes the first hidden state of the decoder
    # decoder_hidden = ....

    # Define the first decoder input. This would essentially be the start_token
    # decoder_input = ....
    decoder_hidden = encoder_last_hidden
            # Define the first decoder input. This would essentially be the start_token
            # decoder_input = ....
    #print (decoder_hidden.size(), "dec size")
    decoder_input = torch.Tensor([start_token for i in range(1)])
    
    decoder_input=decoder_input.unsqueeze(1)
    #print(decoder_input.size(), "size")
            #print("decoder sized 1",decoder_input.size())
    for i in range(max_generated_chars):
        decoder_output, decoder_hidden= decoder.forward(decoder_input, decoder_hidden)
    
        # Calculate the cross entropy between the decoder distribution and Ground truth (current_target)
        # loss += ....
        
        # Find out the most probable character (ni) from the softmax distribution produced
        # ni = ....
        _, indices = torch.max(decoder_output, 1)
        
        indices=indices.unsqueeze(1)
        #print(indices.size(), "size2")
        #indices=torch.unsqueeze(indices,1)
        inp_onehot = torch.FloatTensor(1, vocab_size)
        inp_onehot.zero_()
                #print(inp_onehot.size())
        #print("x size", x.size())
        decoder_input=inp_onehot.scatter_(1, indices.type(torch.LongTensor), 1)
        #print(indices[0][0].item())
        ni = index_to_char[indices[0][0].item()]
        if ni == end_token:
            break
        else:
            gen_string += ni
            
            # update decoder_input at the next time step to be ni 
            # decoder_input = ....

    return gen_string

In [184]:
try:
    train_model(train_dict, val_dict, idx_dict, encoder, decoder, opts)
except KeyboardInterrupt:
    print('Exiting early from training.')

Epoch:   0 | Train loss: 3.172 | Val loss: 3.006 | Gen: eyyyyyyyyyyyyyyyyyyy eaaayyyyyyyyyyyyyyyy eaaayyyyyyyyyyyyyyyy eaaaaayyyyyyyyyyyyyy
Epoch:   1 | Train loss: 2.843 | Val loss: 2.830 | Gen: iaEOSEOSEOSEOSEOSEOSEOSEOSEOSEOSEOSEOSEOSEOSEOSEOSEOSEOS iaaEOSEOSEOSEOSEOSEOSEOSEOSEOSEOSEOSEOSEOSEOSEOSEOSEOS iaaEOSEOSEOSEOSEOSEOSEOSEOSEOSEOSEOSEOSEOSEOSEOSEOSEOS iiaaEOSEOSEOSEOSEOSEOSEOSEOSEOSEOSEOSEOSEOSEOSEOSEOS
Epoch:   2 | Train loss: 2.726 | Val loss: 2.614 | Gen: aayyyyyyyyyyyyyyyyyy eaaaayyyyyyyyyyyyyyy eaaaayyyyyyyyyyyyyyy eaaaaayyyyyyyyyyyyyy
Epoch:   3 | Train loss: 2.472 | Val loss: 2.382 | Gen: ayyyyyyyyyyyyyyyyyyy ooyyyyyyyyyyyyyyyyyy ooyyyyyyyyyyyyyyyyyy oooyyyyyyyyyyyyyyyyy
Epoch:   4 | Train loss: 2.335 | Val loss: 2.190 | Gen: ayyyyyyyyyyyyyyyyyyy ayyyyyyyyyyyyyyyyyyy eiyyyyyyyyyyyyyyyyyy eiiiyyyyyyyyyyyyyyyy
Epoch:   5 | Train loss: 2.185 | Val loss: 2.100 | Gen: ayyyyyyyyyyyyyyyyyyy enyyyyyyyyyyyyyyyyyy enyyyyyyyyyyyyyyyyyy eniiyyyyyyyyyyyyyyyy
Epoch:   6 | Train loss: